**Load the required Packages**


In [6]:
import pandas as pd
import datetime as dt
import numpy as np
import os

**Load the CSV dataset**

In [7]:
df = 









**Run the cell to do Data PreProcessing**

In [8]:
#find the coutry in which each of the customer has made purchase
customer_country=df[['Country','CustomerID']].drop_duplicates()
customer_country
#remove null values
df = df[pd.notnull(df['CustomerID'])]
#removes negative values
df.Quantity.min()
df = df[(df['Quantity']>0)]
df.Quantity.min()
#computes totalprice for monetary calculation
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
NOW = dt.datetime(2011,12,10)
#changing date format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])



**Calculation of Recency, Frequency, Monetary Value**
- Group your customer by Id
- Find the Recency value by subtracting that customer's maximum Invoice Date from NOW
- Find the Frequency by counting the Invoice Number of that customer.
- Find the Monetary value by adding up all the totalprice of purchases of that Customer.


Assign the result to **rfmTable**

**Expected output:**


| CustomerID |recency|frequency |monetary_value|
| ------ | ------ | ------ |------ |
| 12431 | 696 | 3 |105.60 |
| 12583 | 696 | 20 |855.86 |


(Hint: refer your course content)

In [13]:
#calculate the recency, frequency and monetary_value

rfmTable=








**Run the cell to cast your recency into integer value**

In [10]:

rfmTable['recency'] = rfmTable['recency'].astype(int)
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
segmented_rfm = rfmTable


**Run the cell which contains function to calculates the R score**

In [11]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

**Write a function FMScore which calculates the Frequency and Monetary ranking**
- Remember that difference between Rscore and FMScore is in the former the lowest will be assigned 1 whereas in the latter greatest will be assigned 1 rank

In [ ]:
#FMScore calculation
def FMScore(x,p,d):
    
    
    
    
    
    
    
    
    
    
    

**Run the Cell to call the functions R score and FMScore**

In [38]:
#R score calculation

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
# F Score calculation
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
#M score calculation
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))



In [ ]:
R= np.asarray(segmented_rfm[['r_quartile']])
R_mean=np.mean(R);
F= np.asarray(segmented_rfm[['f_quartile']])
F_mean=np.mean(F);
M= np.asarray(segmented_rfm[['m_quartile']])
M_mean=np.mean(M);
final=R_mean+F_mean+M_mean
with open('output1.txt', 'w') as file:
    file.write(str(final))